In [1]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
from tqdm import tqdm

from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import UNETR, SwinUNETR

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)


import torch

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.3
Pytorch version: 2.4.1+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: c:\ProgramData\anaconda3\envs\ship\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.1
Pillow version: 10.2.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: 5.2.0
TorchVision version: 0.19.1+cu121
tqdm version: 4.66.5
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 6.0.0
pandas version: 2.2.3
einops version: 0.8.0
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the 

In [2]:
class_info = {
    0: {"name": "background", "weight": 100},  # weight 없음
    1: {"name": "apo-ferritin", "weight": 62400},
    2: {"name": "beta-amylase", "weight": 1130}, # 4130
    3: {"name": "beta-galactosidase", "weight": 2100}, #3080
    4: {"name": "ribosome", "weight": 1800},
    5: {"name": "thyroglobulin", "weight": 10100},
    6: {"name": "virus-like-particle", "weight": 8400},
}

# 가중치에 비례한 비율 계산
raw_ratios = {
    k: (v["weight"] if v["weight"] is not None else 0.01)  # 가중치 비례, None일 경우 기본값
    for k, v in class_info.items()
}
total = sum(raw_ratios.values())
ratios = {k: v / total for k, v in raw_ratios.items()}

# 최종 합계가 1인지 확인
final_total = sum(ratios.values())
print("클래스 비율:", ratios)
print("최종 합계:", final_total)

# 비율을 리스트로 변환
ratios_list = [ratios[k] for k in sorted(ratios.keys())]
print("클래스 비율 리스트:", ratios_list)

클래스 비율: {0: 0.001162385214460072, 1: 0.725328373823085, 2: 0.013134952923398814, 3: 0.024410089503661515, 4: 0.0209229338602813, 5: 0.11740090666046728, 6: 0.09764035801464606}
최종 합계: 1.0
클래스 비율 리스트: [0.001162385214460072, 0.725328373823085, 0.013134952923398814, 0.024410089503661515, 0.0209229338602813, 0.11740090666046728, 0.09764035801464606]


In [ ]:
from src.dataset.dataset import create_dataloaders
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, NormalizeIntensityd,
    Orientationd, CropForegroundd, GaussianSmoothd, ScaleIntensityd,
    RandSpatialCropd, RandRotate90d, RandFlipd, RandGaussianNoised,
    ToTensord, RandCropByLabelClassesd
)

train_img_dir = "./datasets/train/images"
train_label_dir = "./datasets/train/labels"
val_img_dir = "./datasets/val/images"
val_label_dir = "./datasets/val/labels"
img_depth = 96
img_size = 96  # Match your patch size
n_classes = 7
batch_size = 2 # 13.8GB GPU memory required for 128x128 img size
num_samples = batch_size # 한 이미지에서 뽑을 샘플 수
loader_batch = 1

non_random_transforms = Compose([
    EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
    NormalizeIntensityd(keys="image"),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    GaussianSmoothd(
        keys=["image"],      # 변환을 적용할 키
        sigma=[1.0, 1.0, 1.0]  # 각 축(x, y, z)의 시그마 값
        ),
])
random_transforms = Compose([
    
    RandCropByLabelClassesd(
        keys=["image", "label"],
        label_key="label",
        spatial_size=[img_depth, img_size, img_size],
        num_classes=n_classes,
        num_samples=num_samples, 
        ratios=ratios_list,
    ),
    RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[1, 2]),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
])


In [4]:
train_loader, val_loader = None, None
train_loader, val_loader = create_dataloaders(
    train_img_dir, 
    train_label_dir, 
    val_img_dir, 
    val_label_dir, 
    non_random_transforms = non_random_transforms, 
    random_transforms = random_transforms, 
    batch_size = loader_batch,
    num_workers=0)

Loading dataset: 100%|██████████| 4/4 [00:06<00:00,  1.64s/it]


https://monai.io/model-zoo.html

In [5]:
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import torch
from monai.losses import TverskyLoss
from pathlib import Path
from monai.metrics import DiceMetric
# Model Configuration
start_epoch = 0
best_val_loss = float('inf')

# Training setup
num_epochs = 4000
lamda = 0.52
lr = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrain_path = "./swin_unetr_btcv_segmentation/models/model.pt"
model = SwinUNETR(
    img_size=(img_depth, img_size, img_size),
    in_channels=1,
    out_channels=n_classes,
    feature_size=48,
    use_checkpoint=True,
).to(device)
# Pretrained weights 불러오기
weight = torch.load(pretrain_path, map_location=device)

# 출력 레이어의 키를 제외한 나머지 가중치만 로드
filtered_weights = {k: v for k, v in weight.items() if "out.conv.conv" not in k}

# strict=False로 로드하여 불일치하는 부분 무시
model.load_state_dict(filtered_weights, strict=False)
print("Filtered weights loaded successfully. Output layer will be trained from scratch.")

# Load pretrained weights
# model.load_from(weights=np.load(config_vit.real_pretrained_path, allow_pickle=True))

# TverskyLoss 설정
criterion = TverskyLoss(
    alpha=1- lamda,  # FP에 대한 가중치
    beta= lamda,  # FN에 대한 가중치
    include_background=False,
    softmax=True
)

# 체크포인트 디렉토리 및 파일 설정
checkpoint_base_dir = Path("./model_checkpoints")
checkpoint_dir = checkpoint_base_dir / f"SwinUNETR{img_depth}_{img_size}_lr{lr}_lambda{lamda}_batch{batch_size}"
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)

# 체크포인트 디렉토리 생성
checkpoint_dir.mkdir(parents=True, exist_ok=True)

if checkpoint_dir.exists():
    best_model_path = checkpoint_dir / 'best_model.pt'
    if best_model_path.exists():
        print(f"기존 best model 발견: {best_model_path}")
        try:
            checkpoint = torch.load(best_model_path, map_location=device)
            # 체크포인트 내부 키 검증
            required_keys = ['model_state_dict', 'optimizer_state_dict', 'epoch', 'best_val_loss']
            if all(k in checkpoint for k in required_keys):
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                start_epoch = checkpoint['epoch']
                best_val_loss = checkpoint['best_val_loss']
                print("기존 학습된 가중치를 성공적으로 로드했습니다.")
            else:
                raise ValueError("체크포인트 파일에 필요한 key가 없습니다.")
        except Exception as e:
            print(f"체크포인트 파일을 로드하는 중 오류 발생: {e}")


c:\ProgramData\anaconda3\envs\ship\Lib\site-packages\monai\utils\deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)
C:\Users\Seungwoo\AppData\Local\Temp\ipykernel_38200\2541161994.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects wil

Filtered weights loaded successfully. Output layer will be trained from scratch.


In [6]:
batch = next(iter(val_loader))
images, labels = batch["image"], batch["label"]
print(images.shape, labels.shape)

torch.Size([4, 1, 64, 64, 64]) torch.Size([4, 1, 64, 64, 64])


In [7]:
torch.backends.cudnn.benchmark = True

In [8]:
from monai.metrics import DiceMetric

def create_metric_dict(num_classes):
    """각 클래스별 DiceMetric 생성"""
    metrics = {}
    for i in range(num_classes):
        metrics[f'dice_class_{i}'] = DiceMetric(
            include_background=False if i == 0 else False,
            reduction="mean",
            get_not_nans=False
        )
    return metrics
    
def processing(batch_data, model, criterion, device):
    images = batch_data['image'].to(device)  # Input 이미지 (B, 1, 96, 96, 96)
    labels = batch_data['label'].to(device)  # 라벨 (B, 96, 96, 96)

    labels = labels.squeeze(1)  # (B, 1, 96, 96, 96) → (B, 96, 96, 96)
    labels = labels.long()  # 라벨을 정수형으로 변환

    # 원핫 인코딩 (B, H, W, D) → (B, num_classes, H, W, D)
    
    labels_onehot = torch.nn.functional.one_hot(labels, num_classes=n_classes)
    labels_onehot = labels_onehot.permute(0, 4, 1, 2, 3).float()  # (B, num_classes, H, W, D)

    # 모델 예측
    outputs = model(images)  # outputs: (B, num_classes, H, W, D)

    # Loss 계산
    loss = criterion(outputs, labels_onehot)

    return loss, outputs, labels, outputs.argmax(dim=1)

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0
    with tqdm(train_loader, desc='Training') as pbar:
        for batch_data in pbar:
            optimizer.zero_grad()
            loss, _, _, _ = processing(batch_data, model, criterion, device)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

    return epoch_loss / len(train_loader)

def validate_one_epoch(model, val_loader, criterion, device, epoch, calculate_dice_interval):
    model.eval()
    val_loss = 0
    metrics = create_metric_dict(n_classes)
    class_dice_scores = {i: [] for i in range(n_classes)}
    with torch.no_grad():
        with tqdm(val_loader, desc='Validation') as pbar:
            for batch_data in pbar:
                loss, _, labels, preds = processing(batch_data, model, criterion, device)
                val_loss += loss.item()
                pbar.set_postfix(loss=loss.item())

                # 각 클래스별 Dice 점수 계산
                if epoch % calculate_dice_interval == 0:
                    for i in range(n_classes):
                        pred_i = (preds == i)
                        label_i = (labels == i)
                        dice_score = (2.0 * torch.sum(pred_i & label_i)) / (torch.sum(pred_i) + torch.sum(label_i) + 1e-8)
                        class_dice_scores[i].append(dice_score.item())

    # 각 클래스별 평균 Dice 점수 출력
    if epoch % calculate_dice_interval == 0:
        print("Validation Dice Score")
        for i in range(n_classes):
            mean_dice = np.mean(class_dice_scores[i])
            print(f"Class {i}: {mean_dice:.4f}", end=", ")
            if i == 3:
                print()
        print()

    return val_loss / len(val_loader)

def train_model(
    model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, 
    device, start_epoch, best_val_loss, calculate_dice_interval=1
):
    """
    모델을 학습하고 검증하는 함수
    Args:
        model: 학습할 모델
        train_loader: 학습 데이터 로더
        val_loader: 검증 데이터 로더
        criterion: 손실 함수
        optimizer: 최적화 알고리즘
        num_epochs: 총 학습 epoch 수
        patience: early stopping 기준
        device: GPU/CPU 장치
        start_epoch: 시작 epoch
        best_val_loss: 이전 최적 validation loss
        calculate_dice_interval: Dice 점수 계산 주기
    """

    epochs_no_improve = 0

    for epoch in range(start_epoch, num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # Train One Epoch
        train_loss = train_one_epoch(
            model=model, 
            train_loader=train_loader, 
            criterion=criterion, 
            optimizer=optimizer, 
            device=device
        )

        # Validate One Epoch
        val_loss = validate_one_epoch(
            model=model, 
            val_loader=val_loader, 
            criterion=criterion, 
            device=device, 
            epoch=epoch, 
            calculate_dice_interval=calculate_dice_interval
        )

        print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Early stopping 및 모델 저장 로직
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            checkpoint_path = os.path.join(checkpoint_dir, 'best_model.pt')
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_loss': best_val_loss,
            }, checkpoint_path)
            print(f"========================================================")
            print(f"Best model saved with validation loss: {best_val_loss:.4f}")
            print(f"========================================================")
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print("Early stopping")
            break

train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    patience=50,
    device=device,
    start_epoch=start_epoch,
    best_val_loss=best_val_loss,
    calculate_dice_interval=1
)

Epoch 1/4000


Training:   0%|          | 0/24 [00:00<?, ?it/s]c:\ProgramData\anaconda3\envs\ship\Lib\site-packages\torch\utils\checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]
Validation: 100%|██████████| 4/4 [00:01<00:00,  2.86it/s, loss=0.979]


Validation Dice Score
Class 0: 0.3239, Class 1: 0.1276, Class 2: 0.0000, Class 3: 0.0002, 
Class 4: 0.0175, Class 5: 0.0099, Class 6: 0.0007, 
Training Loss: 0.9849, Validation Loss: 0.9762
Best model saved with validation loss: 0.9762
Epoch 2/4000


Validation: 100%|██████████| 4/4 [00:01<00:00,  2.82it/s, loss=0.943]


Validation Dice Score
Class 0: 0.4436, Class 1: 0.3968, Class 2: 0.0000, Class 3: 0.0004, 
Class 4: 0.0076, Class 5: 0.1083, Class 6: 0.0705, 
Training Loss: 0.9594, Validation Loss: 0.9370
Best model saved with validation loss: 0.9370
Epoch 3/4000


...

Validation Dice Score
Class 0: 0.5251, Class 1: 0.8259, Class 2: 0.1605, Class 3: 0.0009, 
Class 4: 0.0661, Class 5: 0.4462, Class 6: 0.4922, 
Training Loss: 0.8200, Validation Loss: 0.8029
Epoch 129/4000


Validation: 100%|██████████| 4/4 [00:01<00:00,  2.93it/s, loss=0.811]


Validation Dice Score
Class 0: 0.5236, Class 1: 0.8423, Class 2: 0.0776, Class 3: 0.0012, 
Class 4: 0.1047, Class 5: 0.3481, Class 6: 0.5574, 
Training Loss: 0.8134, Validation Loss: 0.7856
Epoch 130/4000


Validation: 100%|██████████| 4/4 [00:01<00:00,  2.96it/s, loss=0.734]

Validation Dice Score
Class 0: 0.5336, Class 1: 0.7957, Class 2: 0.1102, Class 3: 0.0004, 
Class 4: 0.0000, Class 5: 0.3362, Class 6: 0.7115, 
Training Loss: 0.7931, Validation Loss: 0.7784
Early stopping


In [9]:
from monai.inferers import sliding_window_inference
from src.dataset.preprocessing import Preprocessor
from monai.data import Dataset, DataLoader, CacheDataset

config_blob = """{
    "name": "czii_cryoet_mlchallenge_2024",
    "description": "2024 CZII CryoET ML Challenge training data.",
    "version": "1.0.0",

    "pickable_objects": [
        {
            "name": "apo-ferritin",
            "is_particle": true,
            "pdb_id": "4V1W",
            "label": 1,
            "color": [  0, 117, 220, 128],
            "radius": 60,
            "map_threshold": 0.0418
        },
        {
          "name" : "beta-amylase",
            "is_particle": true,
            "pdb_id": "8ZRZ",
            "label": 2,
            "color": [255, 255, 255, 128],
            "radius": 90,
            "map_threshold": 0.0578  
        },
        {
            "name": "beta-galactosidase",
            "is_particle": true,
            "pdb_id": "6X1Q",
            "label": 3,
            "color": [ 76,   0,  92, 128],
            "radius": 90,
            "map_threshold": 0.0578
        },
        {
            "name": "ribosome",
            "is_particle": true,
            "pdb_id": "6EK0",
            "label": 4,
            "color": [  0,  92,  49, 128],
            "radius": 150,
            "map_threshold": 0.0374
        },
        {
            "name": "thyroglobulin",
            "is_particle": true,
            "pdb_id": "6SCJ",
            "label": 5,
            "color": [ 43, 206,  72, 128],
            "radius": 130,
            "map_threshold": 0.0278
        },
        {
            "name": "virus-like-particle",
            "is_particle": true,
            "label": 6,
            "color": [255, 204, 153, 128],
            "radius": 135,
            "map_threshold": 0.201
        },
        {
            "name": "membrane",
            "is_particle": false,
            "label": 8,
            "color": [100, 100, 100, 128]
        },
        {
            "name": "background",
            "is_particle": false,
            "label": 9,
            "color": [10, 150, 200, 128]
        }
    ],

    "overlay_fs_args": {
        "auto_mkdir": true
    },

    "static_root": "./kaggle/input/czii-cryo-et-object-identification/test/static"
    }"""

non_random_transforms = Compose([
    EnsureChannelFirstd(keys=["image"], channel_dim="no_channel"),
    NormalizeIntensityd(keys="image"),
    Orientationd(keys=["image"], axcodes="RAS")
])
model.eval()
preprocessor = Preprocessor(config_blob)
for vol_idx, run in enumerate(preprocessor.root.runs):
    print(f"Processing volume {vol_idx + 1}/{len(preprocessor.root.runs)}")
    tomogram = preprocessor.processing(run=run,task="task")
    task_files = []
    task_files.append({"image": tomogram})    
    task_ds = Dataset(data=task_files, transform=non_random_transforms)
    task_loader = DataLoader(task_ds, batch_size=1, num_workers=0)
    for task_data in task_loader:
        images = task_data['image'].to(device)
        outputs = sliding_window_inference(
            inputs=images,                 # [N, C, D, H, W]
            roi_size=(img_size, img_size, img_size),         # 슬라이딩 윈도우 크기
            sw_batch_size=1,
            predictor=model.forward,
            #overlap=(0.7, 0.25, 0.25)      # Depth 50%, Height 25%, Width 25% 겹침
        )


Config file written to ./kaggle/working/copick.config


ValidationError: 1 validation error for CopickConfigFSSpec
overlay_root
  Field required [type=missing, input_value={'name': 'czii_cryoet_mlc...fig_type': 'filesystem'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing